In [ ]:
# UDP Broadcast Client using asyncio (non-blocking)
# Run this in a separate Jupyter notebook

import socket
import json
import datetime
import uuid
import asyncio
import sys
from IPython.display import clear_output

# Create a unique ID for this client
client_id = str(uuid.uuid4())[:8]

# Store received messages for demonstration purposes
received_messages = []
last_processed_id = -1

# Flag to control execution
running = True

class UDPClient:
    def __init__(self):
        self.transport = None
        
    def connection_made(self, transport):
        self.transport = transport
        print(f"Client {client_id} started, listening on port {client_port}")
        print("Execute the cell again or restart kernel to stop the client")
        
    def datagram_received(self, data, addr):
        global last_processed_id
        
        try:
            server_ip, server_port = addr
            
            # Decode and parse the message
            message_str = data.decode('utf-8')
            message = json.loads(message_str)
            
            # Extract timestamp and state
            timestamp = message.get("timestamp", "unknown")
            state = message.get("state", "unknown")
            message_id = message.get("message_id", -1)
            
            # Only process new messages (in case of duplicates)
            if message_id > last_processed_id:
                last_processed_id = message_id
                
                # Get current time
                receive_time = datetime.datetime.now().isoformat()
                
                # Clear output to avoid long scrolling
                clear_output(wait=True)
                
                # Print client info again
                print(f"Client {client_id} listening on port {client_port}")
                print("Execute the cell again or restart kernel to stop the client")
                
                # Print received message
                print(f"\nReceived broadcast from {server_ip}:{server_port}")
                print(f"Message ID: {message_id}")
                print(f"Timestamp: {timestamp}")
                print(f"State: {state}")
                
                # Append to message history
                received_messages.append({
                    "server_ip": server_ip,
                    "server_port": server_port,
                    "timestamp": timestamp,
                    "receive_time": receive_time,
                    "state": state,
                    "message_id": message_id
                })
                
                # Send response back to the server
                response = f"Client {client_id} received message {message_id}"
                response_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                response_socket.sendto(response.encode('utf-8'), (server_ip, server_port))
                response_socket.close()
                
                # Print statistics
                print(f"Total messages received: {len(received_messages)}")
                
                # Display the last 5 states for demonstration
                if len(received_messages) >= 5:
                    recent_states = [msg["state"] for msg in received_messages[-5:]]
                    print(f"Last 5 states: {recent_states}")
                
        except json.JSONDecodeError:
            print(f"Error: Received invalid JSON data")
        
        except Exception as e:
            print(f"Error: {e}")
    
    def error_received(self, exc):
        print(f"Error received: {exc}")
    
    def connection_lost(self, exc):
        if exc:
            print(f"Connection lost due to error: {exc}")
        else:
            print("Connection closed")

# Define the port clients will listen on
client_port = 37020

# This is the main function that sets up the client
# It's not wrapped in an async function to work better in Jupyter
def start_client():
    # Print initialization message
    print(f"Client {client_id} starting up...")
    
    # Create and configure socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind(('0.0.0.0', client_port))
    
    # Get the current event loop
    loop = asyncio.get_event_loop()
    
    # Create the protocol
    transport, protocol = loop.run_until_complete(
        loop.create_datagram_endpoint(
            lambda: UDPClient(),
            sock=sock
        )
    )
    
    return transport

# Start the client
transport = start_client()

# Note: This approach works well in Jupyter because:
# 1. We're not blocking the cell execution with an infinite loop
# 2. The datagram_received method will be called by the asyncio event loop 
#    even after this cell has "finished" executing
# 3. To stop, just restart the kernel or execute the cell again which will 
#    create a new client